# Acceso a las API de Aemet, y recolección de datos

El objetivo de este Notebook es familiarizarse con la API de AEMET, cuyos datos son distribuidos a través de AEMET OpenData, y realizar una primera inspección de en que forma están estructurados.

https://opendata.aemet.es/dist/index.html


## Tabla de contenidos

<nav>
  <ol>
    <li><a href="#1-API-AEMET">API Aemet y sintaxis del request</a></li>
    <li><a href="#2-acceso-idema">Acceso al "idema"</a></li>
    <li><a href="#3-acceso-EMA">Acceso a la información de las EMA</a></li>
    <li><a href="#4-clima-mensual-anual">Climatologías mensuales anuales</a></li>
    <li><a href="#5-clima-normal-1999-2020">Climatologías normales 1991-2020</a></li>
    <li><a href="#6-valores-extremos">Valores extremos</a></li>  
  </ol>
</nav>

In [2]:
# Importar librerías

import pandas as pd
import requests

## 1 - API Aemet y sintaxis del `request` <a id="1-API-AEMET"></a>

In [1]:
# Cargar la API-key para acceder a los datos de AEMET OpenData

with open('C:/Users/Lander/Documents/API_Keys/apiKey_aemet.txt') as f:
    api_key = f.read()

In [3]:
base_url = 'https://opendata.aemet.es/opendata' # Base API AEMET

querystring = {"api_key": api_key}
headers = {'cache-control': "no-cache"}

## 2 - Acceso al "idema" <a id="2-acceso-idema"></a>

### 2.1 - Necesito los "idema" (Indicativo climatológico de las Estaciones Metereológicas Autónomas) para posteriormente acceder a la información metereológica de estas.

Puedo obtener esta información a través de AEMET OpenData (API REST desarrollado por AEMET que permite la difusión y la reutilización de la información meteorológica y climatológica de la Agencia)


In [4]:
# Inventario de todas las estaciones

end_point = '/api/valores/climatologicos/inventarioestaciones/todasestaciones'

# URL completa
url = base_url + end_point

# Inventario con las características de todas las estaciones climatológicas. 
# Periodicidad: 1 vez al día.response = requests.request("GET", url, headers=headers, params=querystring)
response = requests.request("GET", url, headers=headers, params=querystring)
print(response.status_code)

200


In [5]:
# Inventario con las características de todas las estaciones climatológicas. 
idema_info = response.json()
idema_info

{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/fa4a3c72',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/0556af7a'}

Ha resultado que los datos y los metadatos con los id de las EMAs (Estación Metereológica Automática) están en otra URL, por tanto hay que hacer un nuevo request "GET" a esas direcciones. En concreto la que nos interesa está en la clave "datos".


In [10]:
response = requests.get(idema_info['datos'])

# Visualización de los datos obtenidos por el request
idema = response.json()
idema    


[{'latitud': '394924N',
  'provincia': 'ILLES BALEARS',
  'altitud': '490',
  'indicativo': 'B013X',
  'nombre': 'ESCORCA, LLUC',
  'indsinop': '08304',
  'longitud': '025309E'},
 {'latitud': '394744N',
  'provincia': 'ILLES BALEARS',
  'altitud': '5',
  'indicativo': 'B051A',
  'nombre': 'SÓLLER, PUERTO',
  'indsinop': '08316',
  'longitud': '024129E'},
 {'latitud': '394121N',
  'provincia': 'ILLES BALEARS',
  'altitud': '60',
  'indicativo': 'B087X',
  'nombre': 'BANYALBUFAR',
  'indsinop': '',
  'longitud': '023046E'},
 {'latitud': '393445N',
  'provincia': 'ILLES BALEARS',
  'altitud': '52',
  'indicativo': 'B103B',
  'nombre': 'ANDRATX - SANT ELM',
  'indsinop': '99103',
  'longitud': '022208E'},
 {'latitud': '393305N',
  'provincia': 'ILLES BALEARS',
  'altitud': '50',
  'indicativo': 'B158X',
  'nombre': 'CALVIÀ, ES CAPDELLÀ',
  'indsinop': '',
  'longitud': '022759E'},
 {'latitud': '393319N',
  'provincia': 'ILLES BALEARS',
  'altitud': '3',
  'indicativo': 'B228',
  'nombre': 

"idema" es ahora una lista de diccionarios, que podemos convertir sencillamente en un pandas.DataFrame.
`df_stations` contendrá la información de todas estaciones de España.

In [15]:
df_stations = pd.DataFrame(idema)
df_stations

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
0,394924N,ILLES BALEARS,490,B013X,"ESCORCA, LLUC",08304,025309E
1,394744N,ILLES BALEARS,5,B051A,"SÓLLER, PUERTO",08316,024129E
2,394121N,ILLES BALEARS,60,B087X,BANYALBUFAR,,023046E
3,393445N,ILLES BALEARS,52,B103B,ANDRATX - SANT ELM,99103,022208E
4,393305N,ILLES BALEARS,50,B158X,"CALVIÀ, ES CAPDELLÀ",,022759E
...,...,...,...,...,...,...,...
942,424131N,LLEIDA,2467,9988B,CAP DE VAQUÈIRA,08936,005826E
943,424201N,LLEIDA,1161,9990X,"NAUT ARAN, ARTIES",08107,005237E
944,424634N,LLEIDA,722,9994X,BOSSÒST,,004123E
945,430528N,NAVARRA,334,9995Y,VALCARLOS/LUZAIDE,,011803W


La columna "indicativo" es el denominado "idema" por la propia API de Aemet, así que voy a renombrarla.

In [16]:
df_stations.rename(columns = {'indicativo' : 'idema'}, inplace = True)

### 2.2 - Guardar el dataset en local para poder acceder a él en cualquier momento.

In [18]:
df_stations.to_csv('./data/raw/EMA_info_raw.csv',
                   sep = ',',
                   index = False)

### 2.3 - Información de los metadatos

In [7]:
# Para comprender mejor el significado de cada columna también quiero la información de los metadatos.

# Como ya me conozco la estructura de como almacena AEMET los datos, puedo hacerlo en una única línea de código
df_meta_idema = pd.DataFrame(requests.get(idema_info['metadatos']).json()['campos'])
df_meta_idema

,id,descripcion,tipo_datos,requerido
0,latitud,latitud de la estación,string,True
1,provincia,provincia donde reside la estación,string,True
2,indicativo,indicativo climatológico de la estación,string,True
3,altitud,altitud de la estación,string,True
4,nombre,ubicación de la estación,string,True
5,indsinop,Indicativo sinóptico,string,True
6,longitud,longitud de la estación,string,True


In [8]:
# Guardar la info de los metadatos
df_meta_idema.to_csv('./data/raw/metadatos_EMA.csv',
                     sep = ',',
                     index = False)

### 2.4 - Cargar el DataSet de las estaciones.

In [28]:
df_stations = pd.read_csv('./data/raw/EMA_info_raw.csv')

# Información general del dataset de las estaciones
display(df_stations['provincia'].unique())
display(df_stations.info())
display(df_stations['idema'].nunique())
display(df_stations.duplicated().value_counts())

array(['ILLES BALEARS', 'BALEARES', 'LAS PALMAS', 'STA. CRUZ DE TENERIFE',
       'TARRAGONA', 'BARCELONA', 'GIRONA', 'NAVARRA', 'GIPUZKOA',
       'ARABA/ALAVA', 'BIZKAIA', 'CANTABRIA', 'ASTURIAS', 'LEON', 'LUGO',
       'A CORUÑA', 'PONTEVEDRA', 'OURENSE', 'SORIA', 'BURGOS', 'SEGOVIA',
       'VALLADOLID', 'PALENCIA', 'AVILA', 'MADRID', 'SALAMANCA', 'ZAMORA',
       'GUADALAJARA', 'CUENCA', 'TOLEDO', 'CACERES', 'ALBACETE',
       'CIUDAD REAL', 'BADAJOZ', 'CORDOBA', 'HUELVA', 'CEUTA', 'JAEN',
       'GRANADA', 'ALMERIA', 'SEVILLA', 'CADIZ', 'MELILLA', 'MALAGA',
       'MURCIA', 'ALICANTE', 'VALENCIA', 'TERUEL', 'CASTELLON',
       'LA RIOJA', 'HUESCA', 'ZARAGOZA', 'LLEIDA'], dtype=object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitud    947 non-null    object 
 1   provincia  947 non-null    object 
 2   altitud    947 non-null    int64  
 3   idema      947 non-null    object 
 4   nombre     947 non-null    object 
 5   indsinop   301 non-null    float64
 6   longitud   947 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 51.9+ KB


None

947

False    947
Name: count, dtype: int64

### 2.5 - Tengo una primeras `conclusiones` de este dataset:
- Está bastante limpio, con la mayoría de sus columnas completas
- Tengo el "idema" (único), que era el dato que venía a buscar para poder hacer más `requets` a la API

## 3 - Acceso a la información de las EMA <a id="3-acceso-EMA"></a>

In [30]:
# Ejemplo de acceso a la información de dos estaciones
end_point = '/api/valores/climatologicos/inventarioestaciones/estaciones/B013X,B051A'

url = base_url + end_point

# Información de dos estaciones
response = requests.request("GET", url, headers=headers, params=querystring)
print(response.status_code)
print(response.json())

200
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/d48279d1', 'metadatos': 'https://opendata.aemet.es/opendata/sh/0556af7a'}


De nuevo la información está incrustada en otra URL, por tanto hay que hacer un nuevo "request".
Si toda la información de Aemet está estructurada de esta forma, pensaré en hacer una "request" doble directamente.

In [32]:
# Acceso a los "datos" reales de las dos EMA
response = requests.get(response.json()['datos'])
station_info = response.json()  
station_info

[{'latitud': '394924N',
  'provincia': 'ILLES BALEARS',
  'altitud': '490',
  'indicativo': 'B013X',
  'nombre': 'ESCORCA, LLUC',
  'indsinop': '08304',
  'longitud': '025309E'},
 {'latitud': '394744N',
  'provincia': 'ILLES BALEARS',
  'altitud': '5',
  'indicativo': 'B051A',
  'nombre': 'SÓLLER, PUERTO',
  'indsinop': '08316',
  'longitud': '024129E'}]

### Conclusiones

- La información que tiene este data set es el mismo que ya tengo almacenado en el fichero "EMA_info_raw.csv". Por tanto, este "request" no tiene ningún valor añadido.

## 4 - Acceso a las climatologias mensuales anuales <a id="4-clima-mensual-anual"></a>

Valores medios mensuales y anuales de los datos climatológicos para la estación y el periodo de años pasados por parámetro. Periodicidad de actualización: 1 vez al día.


In [38]:
# Acceso a los datos de una única EMA
end_point = '/api/valores/climatologicos/mensualesanuales/datos/anioini/{anioIniStr}/aniofin/{anioFinStr}/estacion/{idema}'

anioIniStr = '1900'
anioFinStr = '2024'
idema = 'B013X' # idema de Baleares

end_point = end_point.format(anioIniStr = anioIniStr, anioFinStr = anioFinStr, idema = idema)

url = base_url + end_point
url     # No es accesible sin API-Key

'https://opendata.aemet.es/opendata/api/valores/climatologicos/mensualesanuales/datos/anioini/1900/aniofin/2024/estacion/B013X'

In [39]:
response = requests.request("GET", url, headers=headers, params=querystring)
print(response.status_code)
print(response.json())

200
{'descripcion': 'El rango de las fechas no puede ser superior a 36 mesess', 'estado': 404}


### `El rango de las fechas NO puede ser superior a tres meses.`

Por tanto si quiero seguir con esta estrategia debo hacer las consultas iterativamente, o buscar una `nueva estrategia`.

In [45]:
# Acceso a los datos de una única EMA con NUEVO rango de años
end_point = '/api/valores/climatologicos/mensualesanuales/datos/anioini/{anioIniStr}/aniofin/{anioFinStr}/estacion/{idema}'

anioIniStr = '2021'
anioFinStr = '2024'
idema = 'B013X' # idema de Baleares

end_point = end_point.format(anioIniStr = anioIniStr, anioFinStr = anioFinStr, idema = idema)

url = base_url + end_point

response = requests.request("GET", url, headers=headers, params=querystring)
print(response.status_code)
print(response.json())

200
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/47715e04', 'metadatos': 'https://opendata.aemet.es/opendata/sh/997c0034'}


### 4.2 Acceso a los metatados para comprender la información de las columnas

In [47]:
# Acceso a la información de los METADATOS, que describe la información de las columnas
metadatos = requests.get(response.json()['metadatos']).json()
metadatos

{'unidad_generadora': 'Servicio del Banco Nacional de Datos Climatológicos',
 'periodicidad': '1 vez al día',
 'descripcion': 'Climatologías mensuales anuales',
 'formato': 'application/json',
 'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
 'notaLegal': 'https://www.aemet.es/es/nota_legal',
 'campos': [{'id': 'fecha',
   'descripcion': 'año y mes (AAAA-X) donde X es un número del 1 aa 13, indicando el mes y el valor 13 indica valor anual',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'indicativo',
   'descripcion': 'indicativo climatológico',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'nombre',
   'descripcion': 'nombre (ubicación) de la estación',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'provincia',
   'descripcion': 'provincia de la estación',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'altitud',
   'descripcion': 'altitud de la estación en m sobr

In [50]:
# La información de las columnas está en la columna "campos"
df_metadatos = pd.DataFrame(metadatos['campos'])
df_metadatos.head(10)

,id,descripcion,tipo_datos,requerido,unidad
0,fecha,año y mes (AAAA-X) donde X es un número del 1 ...,string,True,NaN
1,indicativo,indicativo climatológico,string,True,NaN
2,nombre,nombre (ubicación) de la estación,string,True,NaN
3,provincia,provincia de la estación,string,True,NaN
4,altitud,altitud de la estación en m sobre el nivel del...,string,True,m
5,tm_mes,Temperatura media mensual/anual,string,False,°C
6,tm_max,Temperatura media mensual/anual de las máximas,string,False,°C
7,tm_min,Temperatura media mensual/anual de las mínimas,string,False,°C
8,ta_max,Temperatura máxima absoluta del mes/año y fecha,string,False,°C
9,ta_min,Temperatura mínima absoluta del mes/año y fecha,string,False,°C


### La información de los metadatos es muy útil. Voy a almacenarla en local también, para poder consultar el signinficado de cada columna en el futuro.

In [51]:
# Guardar la información de los metadatos del clima mensual por anyo
df_metadatos.to_csv('./data/metadatos_clima_mes_anyo.csv',
                    index = False)

### 4.2 - Acceso a los datos de tres años en una única EMA.

In [46]:
# Acceso a los DATOS
B013X_2021_2024 = requests.get(response.json()['datos']).json()
B013X_2021_2024


[{'fecha': '2021-10',
  'indicativo': 'B013X',
  'p_max': '36.3(03)',
  'n_cub': '8',
  'hr': '85',
  'nw_55': '1',
  'tm_min': '10.0',
  'ta_max': '25.4(03)',
  'ts_min': '16.1',
  'nt_30': '0',
  'n_des': '6',
  'w_racha': '23/16.7(30)',
  'np_100': '3',
  'n_nub': '17',
  'nw_91': '0',
  'np_001': '14',
  'ta_min': '5.0(12)',
  'w_rec': '35',
  'e': '150',
  'np_300': '1',
  'p_mes': '107.1',
  'w_med': '1',
  'nt_00': '0',
  'ti_max': '16.2',
  'tm_mes': '15.4',
  'tm_max': '20.8',
  'np_010': '10'},
 {'fecha': '2021-11',
  'indicativo': 'B013X',
  'p_max': '99.1(09)',
  'n_cub': '21',
  'hr': '88',
  'nw_55': '2',
  'tm_min': '8.5',
  'ta_max': '20.8(01)',
  'ts_min': '12.0',
  'nt_30': '0',
  'n_des': '1',
  'w_racha': '14/17.8(29)',
  'np_100': '12',
  'n_nub': '8',
  'nw_91': '0',
  'np_001': '23',
  'ta_min': '3.6(29)',
  'w_rec': '56',
  'e': '116',
  'np_300': '6',
  'p_mes': '502.5',
  'w_med': '2',
  'nt_00': '0',
  'ti_max': '9.4',
  'tm_mes': '10.8',
  'tm_max': '13.2',


In [44]:
df_B013X_2021_2024 = pd.DataFrame(B013X_2021_2024)
display(df_B013X_2021_2024.head())
df_B013X_2021_2024.info()

,fecha,indicativo,p_max,n_cub,hr,nw_55,tm_min,ta_max,ts_min,nt_30,...,w_rec,e,np_300,p_mes,w_med,nt_00,ti_max,tm_mes,tm_max,np_010
0,2021-10,B013X,36.3(03),8,85,1,10.0,25.4(03),16.1,0,...,35,150,1,107.1,1,0,16.2,15.4,20.8,10
1,2021-11,B013X,99.1(09),21,88,2,8.5,20.8(01),12.0,0,...,56,116,6,502.5,2,0,9.4,10.8,13.2,19
2,2021-12,B013X,28.8(02),8,81,NaN,4.4,20.7(31),13.3,0,...,NaN,96,0,59.1,NaN,0,9.8,9.7,15.0,4
3,2021-13,B013X,99.1(09/nov),NaN,74,NaN,9.7,40.5(13/ago),22.8,39,...,NaN,136,13,1235.6,NaN,13,5.4,15.1,20.3,84
4,2021-1,B013X,61.8(10),NaN,80,4,3.0,23.0(28),9.5,0,...,63,84,2,196.5,3,9,5.4,7.6,12.1,12


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   fecha       52 non-null     object
 1   indicativo  52 non-null     object
 2   p_max       39 non-null     object
 3   n_cub       35 non-null     object
 4   hr          45 non-null     object
 5   nw_55       41 non-null     object
 6   tm_min      45 non-null     object
 7   ta_max      45 non-null     object
 8   ts_min      45 non-null     object
 9   nt_30       45 non-null     object
 10  n_des       35 non-null     object
 11  w_racha     41 non-null     object
 12  np_100      39 non-null     object
 13  n_nub       35 non-null     object
 14  nw_91       41 non-null     object
 15  np_001      39 non-null     object
 16  ta_min      45 non-null     object
 17  w_rec       41 non-null     object
 18  e           45 non-null     object
 19  np_300      39 non-null     object
 20  p_mes       

In [56]:
df_B013X_2021_2024.to_csv('./data/raw/clima_mes_B013X_2021_2024.csv',
                          index = False)

## 5 - Climatologias normales 1991-2020 <a id="5-clima-normal-1999-2020"></a>

Valores climatológicos normales (periodo 1991-2020) para la estación pasada por parámetro. Periodicidad: 1 vez al día.

In [69]:
# Climatologías normales sw 1991 a 2020
end_point = '/api/valores/climatologicos/normales/estacion/{idema}'

idema = 'B013X' # idema de Baleares

end_point = end_point.format(idema = idema)

url = base_url + end_point

response = requests.request("GET", url, headers=headers, params=querystring)
print(response.status_code)
print(response.json())

200
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/e9400f61', 'metadatos': 'https://opendata.aemet.es/opendata/sh/98527307'}


In [70]:
# Acceso a la información de los METADATOS, que describe la información de las columnas
metadatos = requests.get(response.json()['metadatos']).json()
metadatos


{'unidad_generadora': 'Servicio del Banco Nacional de Datos Climatológicos',
 'periodicidad': '1 vez al día',
 'descripcion': 'Valores normales para el periodo de referencia 1991-2020',
 'formato': 'application/json',
 'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
 'notaLegal': 'https://www.aemet.es/es/nota_legal',
 'campos': [{'id': 'mes',
   'descripcion': 'mes al que se refieren los datos o 13 de la indicar valor anual',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'indicativo',
   'descripcion': 'indicativo climatológico',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'e_cv',
   'descripcion': 'coeficiente de variación de Pearson de la tensión de vapor media',
   'tipo_datos': 'string',
   'requerido': False},
  {'id': 'e_max',
   'descripcion': 'valor máximo de la tensión de vapor media',
   'tipo_datos': 'string',
   'unidad': '(décimas hPa)',
   'requerido': False},
  {'id': 'e_m

In [71]:
# La información de las columnas está en la columna "campos"
df_metadatos = pd.DataFrame(metadatos['campos'])
df_metadatos.head(10)


,id,descripcion,tipo_datos,requerido,unidad
0,mes,mes al que se refieren los datos o 13 de la in...,string,True,NaN
1,indicativo,indicativo climatológico,string,True,NaN
2,e_cv,coeficiente de variación de Pearson de la tens...,string,False,NaN
3,e_max,valor máximo de la tensión de vapor media,string,False,(décimas hPa)
4,e_md,media aritmética de la tensión de vapor media,string,False,(décimas hPa)
5,e_min,valor mínimo de la tensión de vapor media,string,False,(décimas hPa)
6,e_mn,mediana de la tensión de vapor media,string,False,(décimas hPa)
7,e_n,frecuencia absoluta (nº de años con dato dispo...,string,False,Nº de días
8,e_q1,primer quintil de la tensión de vapor media,string,False,(décimas hPa)
9,e_q2,segundo quintil de la tensión de vapor media,string,False,(décimas hPa)


In [72]:
# Guardar la información de los metadatos de la climatología normal (1991-2020)
df_metadatos.to_csv('./data/raw/metadatos_clima_normales_1991_2020.csv',
                    index = False)

In [73]:
# Acceso a los DATOS
B013X_clima_normal_1991_2020 = requests.get(response.json()['datos']).json()
B013X_clima_normal_1991_2020

[{'indicativo': 'B013X',
  'w_racha_max': '23.1',
  'np_010_n': '24',
  'np_010_s': '3.44',
  'q_max_s': '',
  'n_tor_n': '0',
  'n_tor_s': '',
  'q_max_n': '0',
  'tm_min_q4': '3.3',
  'tm_min_q1': '1.0',
  'tm_min_q3': '2.4',
  'tm_min_q2': '1.8',
  'q_mar_n': '0',
  'n_des_min': '',
  'q_mar_s': '',
  'q_med_n': '0',
  'ts_20_q1': '',
  'ts_20_q2': '',
  'e_cv': '0.09',
  'ts_20_q4': '',
  'e_min': '67.0',
  'np_300_q4': '2.0',
  'np_300_q1': '0.0',
  'np_300_q3': '1.0',
  'hr_max': '84.0',
  'np_300_cv': '1.06',
  'n_nub_max': '',
  'tm_min_cv': '0.63',
  'n_des_mn': '',
  'n_des_md': '',
  'ts_20_s': '',
  'q_med_mn': '',
  'evap_cv': '',
  'q_med_md': '',
  'nt_30_cv': '-',
  'mes': '01',
  'ts_20_cv': '',
  'inso_max': '',
  'ts_20_max': '',
  'np_001_max': '22.0',
  'n_llu_md': '',
  'nv_1000_s': '',
  'ta_min_mn': '-2.5',
  'tm_mes_max': '9.5',
  'ta_max_mn': '18.5',
  'nv_1000_n': '0',
  'ta_max_md': '18.6',
  'nw_91_min': '0.0',
  'ta_max_n': '24',
  'ta_max_s': '2.15',
  't

In [74]:
df_B013X_clima_normal_1991_2020 = pd.DataFrame(B013X_clima_normal_1991_2020)
display(df_B013X_clima_normal_1991_2020)
df_B013X_clima_normal_1991_2020.info()

,indicativo,w_racha_max,np_010_n,np_010_s,q_max_s,n_tor_n,n_tor_s,q_max_n,tm_min_q4,tm_min_q1,...,n_nie_min,nt_30_q4,q_med_q4,n_cub_md,n_cub_mn,ta_max_min,np_010_max,inso_md,inso_mn,evap_mn
0,B013X,23.1,24,3.44,,0,,0,3.3,1.0,...,,0.0,,,,13.8,18.0,,,
1,B013X,23.9,25,4.92,,0,,0,3.0,1.1,...,,0.0,,,,13.1,16.0,,,
2,B013X,20.3,23,2.91,,0,,0,4.7,2.7,...,,0.0,,,,18.1,12.0,,,
3,B013X,19.7,23,2.97,,1,,0,7.3,5.0,...,,0.0,,,,20.8,11.0,,,
4,B013X,16.4,22,2.98,,1,,0,10.2,7.7,...,,2.0,,,,23.8,14.0,,,
5,B013X,19.7,19,1.82,,1,,0,13.7,12.1,...,,8.0,,,,28.7,5.0,,,
6,B013X,21.4,23,0.98,,0,,0,16.6,15.1,...,,19.4,,,,31.6,3.0,,,
7,B013X,18.3,21,2.2,,0,,0,17.4,15.4,...,,22.0,,,,30.0,8.0,,,
8,B013X,41.1,15,2.45,,0,,0,14.3,12.2,...,,4.0,,,,27.1,10.0,,,
9,B013X,20.3,20,3.98,,0,,0,11.3,9.0,...,,0.0,,,,23.0,16.0,,,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Columns: 475 entries, indicativo to evap_mn
dtypes: object(475)
memory usage: 48.4+ KB


In [75]:
df_B013X_clima_normal_1991_2020.to_csv('./data/raw/clima_normal_1991_2020_B013X.csv',
                                       index = False)

### Esta información tiene muchas columnas cuyo siginificado hay que analizar bien antes de poder trabajar con él.

## 6 - Valores extremos <a id="6-valores-extremos"></a>

Valores extremos para la estación y la variable (precipitación, temperatura y viento) pasadas por parámetro. Periodicidad: 1 vez al día.

In [76]:
# Valores extremos para la estación y la variable (precipitación, temperatura y viento) 
# pasadas por parámetro. Periodicidad: 1 vez al día.
end_point = '/api/valores/climatologicos/valoresextremos/parametro/{parametro}/estacion/{idema}'

# Parametro = 'T' (temperatura), 'P' (presión), o 'V' (Viento)
parametro = 'T'
idema = 'B013X' # idema de Baleares

end_point = end_point.format(parametro = parametro, idema = idema)

url = base_url + end_point

response = requests.request("GET", url, headers=headers, params=querystring)
print(response.status_code)
print(response.json())

200
{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/e7855782', 'metadatos': 'https://opendata.aemet.es/opendata/sh/81c232d8'}


In [77]:
# Acceso a la información de los METADATOS, que describe la información de las columnas
metadatos = requests.get(response.json()['metadatos']).json()
metadatos


{'unidad_generadora': 'Servicio del Banco Nacional de Datos Climatológicos',
 'periodicidad': '1 vez al día',
 'descripcion': 'Extremos registrados. Temperaturas',
 'formato': 'json/xml',
 'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
 'notaLegal': 'https://www.aemet.es/es/nota_legal',
 'campos': [{'id': 'indicativo',
   'descripcion': 'Indicativo de la estación',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'nombre',
   'descripcion': 'Nombre de la estación',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'ubicacion',
   'descripcion': 'Ubicación de la estación',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'codigo',
   'descripcion': 'Código de la variable',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'temMin',
   'descripcion': 'Temperatura mínima absoluta de la estación. Contiene 13 campos correspondientes a los 12 meses del año y el último a la mínima ab

In [79]:
# La información de las columnas está en la columna "campos"
df_metadatos = pd.DataFrame(metadatos['campos'])
df_metadatos

,id,descripcion,tipo_datos,requerido,unidad
0,indicativo,Indicativo de la estación,string,True,NaN
1,nombre,Nombre de la estación,string,True,NaN
2,ubicacion,Ubicación de la estación,string,True,NaN
3,codigo,Código de la variable,string,True,NaN
4,temMin,Temperatura mínima absoluta de la estación. Co...,array de string,True,décimas de °C
5,diaMin,Día del mes en que se dio la mínima absoluta d...,array de string,True,NaN
6,anioMin,Año en que se dio la mímina absoluta de la est...,array de string,True,NaN
7,mesMin,Mes en que se dio la mínima absoluta de la est...,string,True,NaN
8,temMax,Temperatura máxima absoluta de la estación. Co...,array de string,True,décimas de °C
9,diaMax,Día del mes en que se dio la máxima absoluta d...,array de string,True,NaN


In [85]:
# Guardar la información de los metadatos de la climatología normal (1991-2020)
df_metadatos.to_csv('./data/raw/metadatos_valores_extremos_T.csv',
                    index = False)

In [81]:
# Acceso a los DATOS
B013X_valores_extremos = requests.get(response.json()['datos']).json()
B013X_valores_extremos

{'indicativo': 'B013X',
 'nombre': 'ESCORCA, LLUC',
 'ubicacion': 'ILLES BALEAR',
 'codigo': '023000',
 'temMin': ['-63',
  '-54',
  '-68',
  '-15',
  '6',
  '63',
  '81',
  '77',
  '57',
  '16',
  '-24',
  '-71',
  '-71'],
 'diaMin': ['26',
  '26',
  '11',
  '1',
  '1',
  '2',
  '7',
  '31',
  '28',
  '27',
  '24',
  '25',
  '25'],
 'anioMin': ['2011',
  '2001',
  '2010',
  '2001',
  '2001',
  '2016',
  '1997',
  '1993',
  '1995',
  '2010',
  '1999',
  '2001',
  '2001'],
 'mesMin': '12',
 'temMax': ['230',
  '232',
  '321',
  '329',
  '363',
  '371',
  '414',
  '414',
  '356',
  '331',
  '263',
  '236',
  '414'],
 'diaMax': ['28',
  '3',
  '23',
  '29',
  '14',
  '25',
  '18',
  '25',
  '4',
  '28',
  '7',
  '14',
  '25'],
 'anioMax': ['2021',
  '2021',
  '2001',
  '2023',
  '2015',
  '2001',
  '2023',
  '2000',
  '2016',
  '1999',
  '2013',
  '2000',
  '2000'],
 'mesMax': '8',
 'temMedBaja': ['56',
  '43',
  '78',
  '101',
  '129',
  '175',
  '211',
  '205',
  '169',
  '136',
  '85',

In [82]:
B013X_valores_extremos = pd.DataFrame(B013X_valores_extremos)
display(B013X_valores_extremos)
B013X_valores_extremos.info()

,indicativo,nombre,ubicacion,codigo,temMin,diaMin,anioMin,mesMin,temMax,diaMax,...,mesMedBaja,temMedAlta,anioMedAlta,mesMedAlta,temMedMin,anioMedMin,mesMedMin,temMedMax,anioMedMax,mesMedMax
0,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,-63,26,2011,12,230,28,...,2,97,2024,7,-2,2002,12,153,2024,7
1,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,-54,26,2001,12,232,3,...,2,105,2021,7,-4,2002,12,166,2020,7
2,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,-68,11,2010,12,321,23,...,2,129,2001,7,17,2003,12,196,2023,7
3,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,-15,1,2001,12,329,29,...,2,141,2023,7,36,1995,12,210,2023,7
4,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,6,1,2001,12,363,14,...,2,184,1999,7,71,2019,12,254,2000,7
5,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,63,2,2016,12,371,25,...,2,234,2022,7,110,2010,12,304,2022,7
6,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,81,7,1997,12,414,18,...,2,262,2023,7,136,1996,12,331,2022,7
7,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,77,31,1993,12,414,25,...,2,255,2023,7,145,2002,12,330,2000,7
8,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,57,28,1995,12,356,4,...,2,218,2021,7,113,1996,12,279,1999,7
9,B013X,"ESCORCA, LLUC",ILLES BALEAR,023000,16,27,2010,12,331,28,...,2,190,2022,7,77,1998,12,253,2022,7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   indicativo   13 non-null     object
 1   nombre       13 non-null     object
 2   ubicacion    13 non-null     object
 3   codigo       13 non-null     object
 4   temMin       13 non-null     object
 5   diaMin       13 non-null     object
 6   anioMin      13 non-null     object
 7   mesMin       13 non-null     object
 8   temMax       13 non-null     object
 9   diaMax       13 non-null     object
 10  anioMax      13 non-null     object
 11  mesMax       13 non-null     object
 12  temMedBaja   13 non-null     object
 13  anioMedBaja  13 non-null     object
 14  mesMedBaja   13 non-null     object
 15  temMedAlta   13 non-null     object
 16  anioMedAlta  13 non-null     object
 17  mesMedAlta   13 non-null     object
 18  temMedMin    13 non-null     object
 19  anioMedMin   13 non-null     ob

In [86]:
B013X_valores_extremos.to_csv('./data/raw/clima_extremo_B013X_T.csv',
                                       index = False)